## Libraries

In [ ]:
# easy ocr 라이브러리
!pip install easyocr -q

# 트랜스포머, 문장 트랜스포머, 데이터셋 라이브러리 설치
!pip install "transformers>=4.42.0" -q
!pip install "sentence-transformers>=2.7.0" -q

# FAISS 설치 (CPU 버전)
!pip install faiss-cpu -q

!pip install rank_bm25 -q
!pip install langchain_openai -q
!pip install langchain -q
!pip install langchain_core -q

# colab - streamlit
!pip install -q gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.8/963.8 kB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.1/292.1 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 2.4 MB/s eta 0:00:00


In [ ]:
import os
import re
import json
import tqdm
import faiss
import easyocr
import numpy as np
import pandas as pd
from typing import List, Optional, Tuple
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer

from langchain_openai import ChatOpenAI
from langchain.tools import StructuredTool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from pydantic import BaseModel, Field
from langchain.agents import create_openai_tools_agent, AgentExecutor

## Drive Mount & Model Loads

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
api_key = #private
os.environ["OPENAI_API_KEY"] = api_key

In [ ]:
#=============================
# RAG DB INDEX, METADATA Load
#=============================
INDEX_PATH = '/content/drive/MyDrive/ICT_project/math_problems.index'
METADATA_PATH = '/content/drive/MyDrive/ICT_project/math_problems_metadata.pkl'

FAISS_INDEX = faiss.read_index(INDEX_PATH)
METADATA = pd.read_pickle(METADATA_PATH).reset_index().rename(columns={"index":"vector_id"})

In [ ]:
#===========================================
# Langchain LLM Model, Embedding Model Load
#===========================================
LLM_MODEL = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
EMB_MODEL = SentenceTransformer("BAAI/bge-m3", device="cuda")

INSTRUCTION = "Represent this math problem for retrieving problems with similar solution concepts: "

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
#=============================================
# BM25(FAISS DB -> BM25), EasyOCR Reader Load
#=============================================
_CORPUS = METADATA["ocr_text"].fillna("").tolist()
_TOKENIZED = [re.findall(r"\w+", t.lower()) for t in _CORPUS]
BM25 = BM25Okapi(_TOKENIZED)

READER = easyocr.Reader(["ko","en"], gpu=True)

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.1% Complete

## Hyperparameters

In [ ]:
#================================
# Hyperparameters for RAG Search
#================================
TOPK_BM25  = 10   # 키워드 후보 수
TOPK_DESC  = 10   # 설명 후보 수
TOPK_IMAGE = 10   # 이미지(OCR) 후보 수

W_BM25 = 0.1      # rerank 시 BM25 가중치
W_DESC = 0.4      # rerank 시 DESC 가중치
W_OCR = 0.5       # rerank 시 OCR 가중치

FINAL_TOPN = 5    # 최종 반환 수

## Tool Definitions

In [ ]:
#===================================
# Util Functions (CRITICAL FUCTIONS)
#===================================
def query_embedding(text: str, add_instruction: bool = True) -> np.ndarray:
    q = INSTRUCTION + text if add_instruction else text
    vector = EMB_MODEL.encode([q], normalize_embeddings=True, convert_to_numpy=True).astype("float32")
    return vector

def faiss_search(qvec: np.ndarray, k: int) -> List[Tuple[int, float, int]]:
    D, I = FAISS_INDEX.search(qvec, k)
    result = []
    for rank, (doc_id, score) in enumerate(zip(I[0], D[0]), 1):
        result.append((int(doc_id), float(score), rank))
    return result

def bm25_search(query: str, k: int) -> List[Tuple[int, float, int]]:
    q_tokens = query.lower().split()
    scores = BM25.get_scores(q_tokens)
    idxs = np.argsort(scores)[::-1][:k]
    result = []
    for rank, idx in enumerate(idxs, 1):
        result.append((int(idx), float(scores[idx]), rank))
    return result

def ocr_image(path: str, min_conf: float = 0.3) -> str:
    try:
        result = READER.readtext(path, detail=1, paragraph=False) # result: [[bbox, text, conf], ...]
        lines = []
        for item in result:
            if not isinstance(item, (list, tuple)) or len(item) < 2:
                continue
            if len(item) >= 3:
                _, text, conf = item[:3]
                if not text or (conf is not None and conf < min_conf):
                    continue
            else:
                _, text = item
                if not text:
                    continue
            lines.append(text)
        txt = " ".join(lines).strip()
        return txt
    except Exception as e:
        print(f"[경고] OCR 실패: {os.path.basename(path)} ({e})")
        return ""

def normalize_scores(scores: List[Tuple[int, float, int]]) -> dict:
    if not scores:
        return {}
    # minmax
    arr = np.array([s for _, s, _ in scores], dtype=np.float32)
    min_v, max_v = float(arr.min()), float(arr.max())
    if max_v - min_v < 1e-9:
        norm = np.zeros_like(arr)
    else:
        norm = (arr - min_v) / (max_v - min_v)
    normalized_scores = {int(doc_id): float(ns) for (doc_id, _, _), ns in zip(scores, norm)}
    return normalized_scores

def renorm_weights(has_desc, has_ocr, has_bm25,
                    w_desc=0.5, w_ocr=0.4, w_bm25=0.1):
    pairs = []
    if has_desc:  pairs.append(("desc",  w_desc))
    if has_ocr:   pairs.append(("ocr",   w_ocr))
    if has_bm25:  pairs.append(("bm25",  w_bm25))

    if not pairs:
        return {"desc":0.0,"ocr":0.0,"bm25":0.0}

    # 입력된 개수 만큼의 가중치를 renormalize 하여 반환
    s = sum(w for _, w in pairs)
    renormalized_weights = {k: (w/s) for k, w in pairs}
    return renormalized_weights

def combsum_merge_adaptive(
    desc_scores: List[Tuple[int, float, int]],
    ocr_scores:  List[Tuple[int, float, int]],
    bm25_scores: List[Tuple[int, float, int]],
    w_desc: float = W_DESC,
    w_ocr:  float = W_OCR,
    w_bm25: float = W_BM25,
    final_topn: int = FINAL_TOPN
) -> List[int]:
    desc_map = {doc_id:score for doc_id,score,_ in (desc_scores or [])}
    ocr_map  = {doc_id:score for doc_id,score,_ in (ocr_scores  or [])}
    bm25_map = normalize_scores(bm25_scores or [])

    has_desc  = len(desc_map)  > 0
    has_ocr   = len(ocr_map)   > 0
    has_bm25  = len(bm25_map)  > 0
    eff = renorm_weights(has_desc, has_ocr, has_bm25, w_desc, w_ocr, w_bm25)

    doc_ids = set(desc_map) | set(ocr_map) | set(bm25_map)

    if not doc_ids:
        return list(range(min(final_topn, len(METADATA))))

    combined = []
    for doc in doc_ids:
        score = 0.0
        score += eff.get("desc", 0.0)  * desc_map.get(doc, 0.0)
        score += eff.get("ocr",  0.0)  * ocr_map.get(doc,  0.0)
        score += eff.get("bm25", 0.0)  * bm25_map.get(doc, 0.0)
        combined.append((doc, score))
    combined.sort(key=lambda x: x[1], reverse=True)
    topn_docs = [doc_id for doc_id, _ in combined[:final_topn]]
    return topn_docs

In [ ]:
#=====================================================
# LLM Agent Tool Definitions
# - StructuredTool library
# - args: func, name, description, args_schema(Class)
#=====================================================

# 1st Tool: Search_BM25

def search_bm25_tool(keywords: str, k: int = TOPK_BM25) -> str:
    results = bm25_search(keywords, k=k)
    return json.dumps(results, ensure_ascii=False)

class BM25_Input_Schema(BaseModel):
    keywords: str = Field(..., description="짧은 키워드(단원명 또는 개념/유형) ex: '조건부확률, 배반사건'")
    k: int = Field(default=TOPK_BM25, description="상위 몇 개의 결과를 가져올지")

Search_BM25 = StructuredTool.from_function(
    func=search_bm25_tool,
    name="Search_BM25",
    description="사용자의 KEYWORDS 입력이 있으면 실행: 수학문제DB로부터 BM25 단어 기반의 검색",
    args_schema=BM25_Input_Schema
)

# 2nd Tool: Search_DESC

def search_desc_tool(description: str, k: int = TOPK_DESC) -> str:
    qvec = query_embedding(description, add_instruction=True)
    results = faiss_search(qvec, k=k)
    return json.dumps(results, ensure_ascii=False)

class Desc_Input_Schema(BaseModel):
    description: str = Field(..., description="문제 설명, 풀이, 또는 접근법 ex: '같은 것이 있는 순열을 이용한 최단거리 계산 문제'")
    k: int = Field(default=TOPK_DESC, description="상위 몇 개의 결과를 가져올지")

Search_DESC = StructuredTool.from_function(
    func=search_desc_tool,
    name="Search_DESC",
    description="사용자의 DESCRIPTION 입력이 있으면 실행: 수학문제DB로부터 BGE‑M3 embedding 의미기반의 검색",
    args_schema=Desc_Input_Schema
)

# 3rd Tool: Search_IMAGE

def search_image_tool(image_path: str, k: int = TOPK_IMAGE) -> str:
    if not image_path or not os.path.exists(image_path):
        return json.dumps([])
    q_text = ocr_image(image_path, min_conf=0.3)
    if not q_text:
        return json.dumps([])
    qvec = query_embedding(q_text, add_instruction=True)
    results = faiss_search(qvec, k=k)
    return json.dumps(results, ensure_ascii=False)

class IMAGE_Input_Schema(BaseModel):
    image_path: str = Field(..., description="수학문제 스크린샷(캡처)의 파일 경로")
    k: int = Field(default=TOPK_IMAGE, description="상위 몇 개의 결과를 가져올지")

Search_IMAGE = StructuredTool.from_function(
    func=search_image_tool,
    name="Search_IMAGE",
    description="사용자의 IMAGE 입력이 있으면 실행: 이미지 OCR 처리 후 수학문제DB로부터 BGE‑M3 embedding 의미기반의 검색",
    args_schema=IMAGE_Input_Schema
)

# 4th Tool: Combine_Merge

def combsum_merge_tool(
    desc_results_json: Optional[str],
    ocr_results_json:  Optional[str],
    bm25_results_json: Optional[str],
    final_topn: int = FINAL_TOPN,
    w_desc: float = W_DESC,
    w_ocr:  float = W_OCR,
    w_bm25: float = W_BM25
) -> str:
    # JSON -> Python list 파싱
    def _loads(js):
        if not js:
            return []
        try:
            arr = json.loads(js)  # [(doc_id, score, rank), ...]
            out = []
            for t in arr:
                doc_id, score, rank = int(t[0]), float(t[1]), int(t[2])
                out.append((doc_id, score, rank))
            return out
        except Exception:
            return []

    desc_scores = _loads(desc_results_json)
    ocr_scores  = _loads(ocr_results_json)
    bm25_scores = _loads(bm25_results_json)

    # Util의 combsum_merge_adaptive 사용 (BM25는 min-max 정규화, 없는 신호는 자동 재분배)
    top_ids = combsum_merge_adaptive(
        desc_scores=desc_scores,
        ocr_scores=ocr_scores,
        bm25_scores=bm25_scores,
        w_desc=w_desc,
        w_ocr=w_ocr,
        w_bm25=w_bm25,
        final_topn=final_topn
    )

    # 최종 문서 JSON 반환 (LLM이 문제정보를 답변할 수 있게)
    docs = METADATA.iloc[top_ids][["original_filename", "ocr_text"]].copy().reset_index(drop=True)
    docs.insert(0, "rank", range(1, len(docs)+1))
    return docs.to_json(orient="records", force_ascii=False)

class JSON_HYPER_Input_Schema(BaseModel):
    desc_results_json: Optional[str] = Field(None, description="Search_DESC 결과")
    ocr_results_json:  Optional[str] = Field(None, description="Search_IMAGE 결과")
    bm25_results_json: Optional[str] = Field(None, description="Search_BM25 결과")
    final_topn: int = Field(default=FINAL_TOPN, description="Search 결과 최종 반환 개수")
    w_desc: float = Field(default=W_DESC, description="DESCRIPTION 임베딩 가중치")
    w_ocr:  float = Field(default=W_OCR,  description="IMAGE OCR 임베딩 가중치")
    w_bm25: float = Field(default=W_BM25, description="BM25 가중치")

Combine_Merge = StructuredTool.from_function(
    func=combsum_merge_tool,
    name="Combine_Merge",
    description="있는 신호끼리 비율을 재분배한 CombSUM(정규화+가중합, BM25는 min-max)으로 최종 상위 문서를 JSON으로 반환.",
    args_schema=JSON_HYPER_Input_Schema
)

## LLM Agent

In [ ]:
#===========================================
# LLM Agent Setting with Prompt Engineering
#===========================================

TOOLS = [Search_BM25, Search_DESC, Search_IMAGE, Combine_Merge]

SYSTEM_PROMPT = """\
당신은 확률과 통계 과목의 유사문제 추출 전문가입니다.

사용자의 입력 프롬프트는 아래와 같은 문자열입니다.
"키워드:KEYWORDS, 설명:DESCRIPTION, 이미지경로:IMAGE_PATH"
사용자는 찾고자 하는 유사문제에 대한 정보를 위의 3가지 중 선택적으로 입력합니다.
만약 사용자가 입력하지 않은 항목이 있다면 None으로 기록됩니다.

다음 지침을 반드시 따르세요.
1. 사용자의 선택적 입력을 인식하고 분석하여 필요한 Tool을 호출하세요.
- KEYWORDS 있으면 Search_BM25 호출
- DESCRIPTION 있으면 Search_DESC 호출
- IMAGE_PATH 있으면 Search_IMAGE 호출

2. 최종 결합은 반드시 Combine_Merge Tool을 호출하세요.
- Combine_Merge 호출할 때 final_topn은 FINAL_TOPN으로 설정하세요.
- 반드시 최종 답변에 이 개수가 정확히 반영되어야 합니다.

3. 최종 응답형식은 오직 JSON 배열의 string만 출력하세요.
- 배열에는 각 유사문제의 파일명(original_filename)만 포함
- ex: ["파일명1.png", "파일명2.png"]
"""

In [ ]:
def fix_json_string(json_str: str) -> str:
    """
    JSON 문자열 안에 포함된 에티오픽 문자 'ዋ'(U+12CB)를
    한글 '의'(U+C758)로 교체해서 반환
    """
    if not isinstance(json_str, str):
        raise ValueError("입력은 JSON 문자열이어야 합니다.")
    return json_str.replace("ዋ", "의")

## UI Interface

In [ ]:
import gradio as gr
import os
import ast

In [ ]:
os.makedirs("similar_problems", exist_ok=True)

PROMPT = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

AGENT = create_openai_tools_agent(LLM_MODEL, TOOLS, PROMPT)
EXECUTOR = AgentExecutor(agent=AGENT, tools=TOOLS)

def LLM_Agent(keywords: str = None, description: str = None, image_path: str = None):
    BASE_GDRIVE_PATH = '/content/drive/MyDrive/ICT_project/datasets/math_problem/'
    USER_PROMPT = f"키워드:{keywords}, 설명:{description}, 이미지경로:'{image_path}'"
    try:
        output = EXECUTOR.invoke({"input": USER_PROMPT})['output']

        # ✨ 1. 디버깅을 위해 LLM의 원본 출력을 항상 확인합니다.
        print(f"LLM 원본 출력: {output}")

        if not output or not output.strip():
            print("LLM Agent가 빈 결과를 반환했습니다.")
            return []

        # ✨ 2. 정규 표현식으로 리스트([...]) 또는 튜플(...) 형태의 문자열을 찾습니다.
        # re.search()는 문자열 전체에서 패턴과 일치하는 첫 번째 부분을 찾습니다.
        match = re.search(r'(\[.*?\]|\(.*?\))', output)

        # 3. 매칭된 부분이 있는지 확인합니다.
        if match:
            # 매칭된 문자열(예: "['a.png', 'b.png']")을 추출합니다.
            extracted_text = match.group(0)

            # 4. 추출된 문자열을 파싱합니다.
            file_list = ast.literal_eval(extracted_text)

            if not isinstance(file_list, (list, tuple)):
                print(f"오류: 추출된 결과가 리스트 또는 튜플이 아닙니다. (결과: {file_list})")
                return []

            full_path_list = [os.path.join(BASE_GDRIVE_PATH, filename) for filename in file_list]
            print(f"Gradio에 전달될 최종 경로 리스트: {full_path_list}")
            return full_path_list
        else:
            # 5. 문장 속에서 리스트나 튜플을 찾지 못했을 경우
            print("LLM의 응답에서 파일 목록을 찾을 수 없습니다.")
            return []

    except Exception as e:
        print(f"LLM Agent 실행 중 오류 발생: {e}")
        return []

def loading_text():
    return gr.Button("유사문제 찾는중...", interactive=False)

def complete_text():
    return gr.Button("완료", interactive=True)

with gr.Blocks() as demo:
    gr.Markdown("# 고등수학 유사문제 추출 프로그램")
    gr.Markdown("키워드, 설명, 이미지 중 하나 이상을 입력하고 '유사문제 찾기' 버튼을 누르세요.")

    with gr.Row():
        with gr.Column():
            keywords_input = gr.Textbox(lines=1, placeholder="예: 조건부확률, 중복조합", label="핵심 키워드")
            description_input = gr.Textbox(lines=3, placeholder="예: 같은 것이 있는 순열을 이용한 최단거리 계산 문제", label="문제 설명")
            image_input = gr.Image(type="filepath", label="문제 이미지")
            submit_btn = gr.Button("유사문제 찾기", variant="primary")
        with gr.Column():
            # 💡 출력 컴포넌트를 gr.Gallery로 변경!
            output_gallery = gr.Gallery(
                label="유사문제 추출 결과",
                show_label=True,
                columns=1, # 이미지를 세로로 1줄로 나열
                object_fit="contain",
                height="auto"
            )

    (
        submit_btn.click(loading_text, outputs=submit_btn)
      .then(
          fn=LLM_Agent,
          inputs=[keywords_input, description_input, image_input],
          outputs=output_gallery
      )
      .then(complete_text, outputs=submit_btn)
    )


# UI 실행 (share=True 옵션으로 외부 접속 URL 생성)
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://6c58a287576e4f81b8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


LLM 원본 출력: ```json
["2026 수능특강 03.확률의뜻과활용 level1_7.PNG","2024 수능특강 03.확률의뜻과활용 level1_2.PNG","2026 수능특강 03.확률의뜻과활용 level2_7.PNG","2024 수능특강 03.확률의뜻과활용 level2_2.PNG","2025 수능특강 03.확률의뜻과활용 유제2.PNG"]
```
Gradio에 전달될 최종 경로 리스트: ['/content/drive/MyDrive/ICT_project/datasets/math_problem/2026 수능특강 03.확률의뜻과활용 level1_7.PNG', '/content/drive/MyDrive/ICT_project/datasets/math_problem/2024 수능특강 03.확률의뜻과활용 level1_2.PNG', '/content/drive/MyDrive/ICT_project/datasets/math_problem/2026 수능특강 03.확률의뜻과활용 level2_7.PNG', '/content/drive/MyDrive/ICT_project/datasets/math_problem/2024 수능특강 03.확률의뜻과활용 level2_2.PNG', '/content/drive/MyDrive/ICT_project/datasets/math_problem/2025 수능특강 03.확률의뜻과활용 유제2.PNG']
LLM 원본 출력: ```json
["2026 수능특강 03.확률의뜻과활용 level1_7.PNG","2024 수능특강 03.확률의뜻과활용 l